# Imports

In [1]:
import spatial2
import utils
import anndata as ad
import pandas as pd
import os

# Load AnnData

In [2]:
# load main adata
adata = ad.read_h5ad('./4_h5ad_files/adata_fov_integrated_spoox.h5ad')

/Users/joaoluizsfilho/opt/anaconda3/envs/Squidpy/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
adata.obs

In [ ]:
adata.obs['sample_ID']

In [ ]:
# Adding HIV status 
anno = pd.read_csv(filepath_or_buffer='./cellID.csv', index_col=0)

In [ ]:
anno.index

In [ ]:
adata.obs.index

In [ ]:
adata.obs['Master_Index'] = anno.index

In [ ]:
adata.obs.index = adata.obs['Master_Index']

In [ ]:
adata.obs

In [ ]:
adata.write('./4_h5ad_files/adata_fov_integrated_spoox.h5ad')

# <font color=blue>SpOOx pipeline</font>

This will run the 'spatial statistics' component of the SpOOx analysis pipeline on an AnnData object. You can find more information here: https://github.com/Taylor-CCB-Group/SpOOx/tree/main/src/spatialstats

This can take quite a while to run, depending on the number of ROIs

Any errors in running functions will be saved in _errors.csv_

There several extra parameters - run _spatial.run_spoox?_ to see the function help

## Parameters

##### adata
    AnnData object, or path (string) to a saved h5ad file.
##### population_obs
    The .obs that defines the population for each cell    
##### groupby
    If specifed, should be a .obs that identifies different groups in the data.
    In the summary step, it will then summarise the analyses at the levels of the different groups.     
##### samples
    Specify a list of samples, by default will process all samples    
##### specify_functions
    By default will run the follow functions from the Spoox pipeline: 
    paircorrelationfunction morueta-holme networkstatistics 
    
    This is a complete list that will be run if 'all' is used: 
        paircorrelationfunction
        localclusteringheatmaps
        celllocationmap
        contourplots
        quadratcounts
        quadratcelldistributions
        morueta-holme
        networkstatistics        
##### run_analysis
    Whether or not to run the analyis, or just create the spatialstats file    
##### analyse_samples_together
    Whether to analyse all samples together    
##### summary
    Whether to run summary script

## Returns
Creates two folders with outputs of SpOOx pipeline - **spooxout** and **spooxout_summary** by default
        


In [ ]:
# edit function
# create spooxout and spooxout_summary directories
spatial2.run_spoox(adata, 
                  population_obs='CIPR_ordered_clusters', #Define which population to use from adata.obs
                  groupby = 'Epithelial_cluster', #Which column in the adata.obs that defines the groups
                  specify_functions = 'paircorrelationfunction morueta-holme' , masks_source_directory=None)

# Load in SpOOx summary data

In [ ]:
data = pd.read_csv('spooxout_summary/summary.tsv', sep='\t') #Location to the 'summary.tsv' that Spoox makes

# Heatmaps of SpOOx outputs

In [ ]:
spatial2.create_spoox_heatmaps?

In [ ]:
import os
spatial2.create_spoox_heatmaps(data)

# Adjacency cell networks

In [ ]:
spatial.create_network_graphs?

In [ ]:
spatial2.create_network_graphs(data, fig_size=(10,10), layout_type='circular')

## Plot specifc pop of interest

In [ ]:
spatial2.create_network_graphs(
    data, 
    fig_size=(3,3), 
    edge_scale=2, 
    draw_labels=True,
    figure_box=False, filters={'MH_FDR_less': 0.05},
    add_legend=False, edge_color_column='MH_SES',
    center_cell_population='Activated Endothelial cell',
    layout_type='kamada_kawai',
)

## Plot avoidance and enrichment
This doesn't add a lot extra over the MH data alone, as the GR / PCF data is not used to filter the data

In [ ]:
spatial2.create_network_graphs(data, fig_size=(10,10),
                             filters={'MH_FDR_less': 0.05},
                             edge_color_column='MH_SES',
                             edge_color_map='coolwarm', layout_type='circular')   